In [ ]:
# %tensorflow_version 1.x
!apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install Cython | grep Successfully 
from google.colab import drive
drive.mount('/content/gdrive')

In [3]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3693, done.
remote: Counting objects: 100% (3693/3693), done.
remote: Compressing objects: 100% (3080/3080), done.
remote: Total 3693 (delta 989), reused 1508 (delta 562), pack-reused 0
Receiving objects: 100% (3693/3693), 48.76 MiB | 14.32 MiB/s, done.
Resolving deltas: 100% (989/989), done.


In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cd /content
git clone https://github.com/cocodataset/cocoapi.git
cd /content/cocoapi/PythonAPI
make
cp -r pycocotools /content/models/research/
cd /content/models/research/
cp object_detection/packages/tf2/setup.py .
python -m pip install .
cd /content

In [ ]:
# %cd /content
# !git clone https://github.com/tensorflow/models.git
# !git clone https://github.com/cocodataset/cocoapi.git
# %cd /content/cocoapi/PythonAPI 
# !make 
# !cp -r pycocotools /content/models/research/
# %cd /content/models/research
# %set_env PYTHONPATH=/content/models/research:/content/models/research/slim
# !protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim
%cd /content/models/research
!python object_detection/builders/model_builder_test.py

In [7]:
%cd /content/models/research/object_detection

/content/models/research/object_detection


In [8]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

In [28]:
from tensorflow.keras.models import Sequential

In [9]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

In [10]:
# This is needed to display the images.
%matplotlib inline
from utils import label_map_util

from utils import visualization_utils as vis_util

In [11]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

In [12]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [15]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [16]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [19]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]


# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [20]:
!pwd

/content/models/research/object_detection


In [21]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [29]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.compat.v1.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict



In [30]:
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

AttributeError: ignored

In [ ]:
!rm -r /content/annotations
!rm -r /content/coco

In [ ]:
%cd /content
!mkdir coco
%cd /content/coco
!wget -q http://images.cocodataset.org/zips/train2014.zip
!unzip -qq train2014.zip 
!ls
!rm train2014.zip
!wget -q http://images.cocodataset.org/zips/val2014.zip
!unzip -qq val2014.zip
!ls
!rm val2014.zip
!wget -q http://images.cocodataset.org/zips/test2014.zip
!unzip -qq test2014.zip
!ls
!rm test2014.zip
!wget -q http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!unzip -qq annotations_trainval2014.zip
!ls
!rm annotations_trainval2014.zip
!wget -q http://images.cocodataset.org/annotations/image_info_test2014.zip
!unzip -qq image_info_test2014.zip
!ls
!rm image_info_test2014.zip

/content
mkdir: cannot create directory ‘coco’: File exists
/content/coco
image_info_test2014.zip  train2014  train2014.zip


In [ ]:
%cd /content/coco
!rm -rf tfrecord
!mkdir tfrecord
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

/content/coco
env: PYTHONPATH=/content/models/research:/content/models/research/slim


In [ ]:
%cd /content/models/research/object_detection/dataset_tools
!python create_coco_tf_record.py --logtostderr \
      --train_image_dir=/content/coco/train2014/ \
      --val_image_dir=/content/coco/val2014/ \
      --test_image_dir=/content/coco/test2014/ \
      --train_annotations_file=/content/coco/annotations/instances_train2014.json \
      --val_annotations_file=/content/coco/annotations/instances_val2014.json \
      --testdev_annotations_file=/content/coco/annotations/image_info_test2014.json \
      --output_dir=/content/coco/tfrecord | grep Finished

In [ ]:
!pwd
%cd /content/coco/tfrecord
!ls

In [ ]:
%cd /content/coco
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -xzvf ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!ls /content/coco/ssd_mobilenet_v2_coco_2018_03_29/

In [ ]:
%cd /content/coco
!mkdir config_dir
%cd /content/coco/config_dir
!wget https://raw.githubusercontent.com/JinFree/models/master/ssd_inceptionv2_in_colab.config
!ls /content/coco/config_dir

In [ ]:
%cd /content/models/research
!python object_detection/model_main.py \
    --pipeline_config_path=/content/coco/config_dir/ssd_inceptionv2_in_colab.config \
    --model_dir=/content/coco/config_dir/ \
    --num_train_steps=50000 \
    --sample_1_of_n_eval_examples=1 \
    --alsologtostderr

In [ ]:
!ls /content/coco/config_dir/

In [ ]:
%cd /content/coco/
!rm -rf ssd_mobilenet_v2_coco_2018_03_29
!mkdir ssd_mobilenet_v2_coco_2018_03_29
%cd /content/models/research
!python object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/coco/config_dir/ssd_inceptionv2_in_colab.config \
    --trained_checkpoint_prefix=/content/coco/config_dir/model.ckpt-485 \
    --output_directory=/content/coco/ssd_mobilenet_v2_coco_2018_03_29/
!ls /content/coco/ssd_mobilenet_v2_coco_2018_03_29/

In [ ]:
!rm -rf /content/coco/config_dir
!mkdir /content/coco/config_dir
%cd /content/coco/config_dir
!wget https://raw.githubusercontent.com/JinFree/models/master/ssd_inceptionv2_in_colab.config

In [ ]:
%cd  /content/gdrive/'My Drive'/
!rm -rf tf_trained
!mkdir tf_trained
!cp -r /content/coco/ssd_mobilenet_v2_coco_2018_03_29 /content/gdrive/'My Drive'/tf_trained
!ls /content/gdrive/'My Drive'/tf_trained/ssd_mobilenet_v2_coco_2018_03_29